In [1]:
import numpy as np

# 1


In [2]:
def _simplex_1p(A, c, bs_i, eps=1e-6):
    """
    max. c^T x
    s.t. A[:][:-1] x == A[:][-1], x >= 0, A[i][bs_i[i]] != 0, x[!bs_i] == 0
    returns x[i] < 0: x[i] = inf
    """
    m, n = A.shape
    assert (m < n) and (c.shape == n,) and (len(bs_i) == m)
    x = np.zeros(n - 1)
    for i in range(m):
        assert (0 <= bs_i[i] < (n - 1)) and abs(A[i][bs_i[i]]) > eps
        A[i] /= A[i][bs_i[i]]
        c -= c[bs_i[i]] * A[i]
        for j in range(m):
            if i != j:
                A[j] -= A[j][bs_i[i]] * A[i]
    for i in range(m):
        assert A[i][-1] >= -eps
        x[bs_i[i]] = A[i][-1]
    while c[idx1 := np.argmin(c[:-1])] < -eps:
        x[idx1] = np.inf
        idx2 = -1
        for i in range(m):
            if A[i][idx1] > eps:
                tmp = A[i][-1] / A[i][idx1]
                if x[idx1] - tmp > eps:
                    idx2 = i
                    x[idx1] = tmp
            elif (idx2 < 0) and (A[i][idx1] < -eps):
                idx2 = i
        if idx2 >= 0:
            if x[idx1] == np.inf:
                x[idx1] = -1
                for i in range(m):
                    if A[i][idx1] < -eps:
                        x[bs_i[i]] = A[i][idx1]
            else:
                x[bs_i[idx2]] = 0
                bs_i[idx2] = idx1
                A[idx2] /= A[idx2][idx1]
                c -= c[idx1] * A[idx2]
                for i in range(m):
                    if i != idx2:
                        A[i] -= A[i][idx1] * A[idx2]
                        x[bs_i[i]] = A[i][-1]
        if x[idx1] < -eps:
            c[-1] = np.inf
            break
    return c[-1], x

In [3]:
def simplex(A, b, c, eps=1e-6):
    """
    max. c^T x
    s.t. A x <= b, x >= 0
    returns x[i] < 0: x[i] = inf
    """
    m, n = A.shape
    assert (c.shape == n,) and (b.shape == m,)
    A_ = np.concatenate((A, np.eye(m), b.reshape(-1, 1)), axis=1)
    c_ = np.concatenate((-c, np.zeros(m + 1)))
    bs_i = list(range(n, n + m))
    a_i = []
    for i in range(m):
        if A_[i][-1] < -eps:
            a_i.append(i)
    if a_i:
        l = len(a_i)
        A_ = np.concatenate((np.zeros((m, l)), A_), axis=1)
        for i in range(m):
            bs_i[i] += l
        for i in range(l):
            A_[a_i[i]][i] = -1
            bs_i[a_i[i]] = i
        res = _simplex_1p(
            A_, np.concatenate((np.ones(l), np.zeros(n + m + 1))), bs_i, eps
        )
        if abs(res[0]) > eps:
            return None, None
        else:
            A_ = A_[:, l:]
            for i in range(m):
                bs_i[i] -= l
    res = _simplex_1p(A_, c_, bs_i, eps)
    return res[0], res[1][:n]

In [4]:
A = np.array([[2, 1], [1, 1], [1, 0]])
b = np.array([20, 18, 8])
c = np.array([7, 4])
display(simplex(A, b, c))

(np.float64(78.0), array([ 2., 16.]))

In [5]:
A = np.array([[-3, -2], [-1, -4], [1, 1]])
b = np.array([-3, -4, 5])
c = np.array([5, 8])
display(simplex(A, b, c))

(np.float64(40.0), array([0., 5.]))

In [6]:
A = np.array([[1, 1, -3], [-4, 1, -1], [-2, -3, 1]])
b = np.array([8, -2, -4])
c = np.array([-2, 1, -1])
display(simplex(A, b, c))

(np.float64(inf), array([-0.4, -2.6, -1. ]))

In [7]:
A = np.array([[2, 1], [-1, -1]])
b = np.array([2, -5])
c = np.array([3, 1])
display(simplex(A, b, c))

(None, None)